<a href="https://colab.research.google.com/github/SumathiGit/Defect-Detection-Food-Package/blob/main/Food_Defect_Detection_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
""" Making Prediction """

' Making Prediction '

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
labels = ['Label1_Beans', 'Label1_Beverage', 'Label1_Biscuits', 'Label1_Bread',
       'Label1_CAKE', 'Label1_Candy', 'Label1_Cans', 'Label1_Cereal',
       'Label1_Chips', 'Label1_ChipsPack', 'Label1_Chocolate',
       'Label2_BeansDefected', 'Label2_BeansPerfect',
       'Label2_BeverageDefected', 'Label2_BeveragePerfect',
       'Label2_BiscuitDefected', 'Label2_BiscuitPefect',
       'Label2_BreadDefected', 'Label2_BreadPerfect', 'Label2_CakeDefected',
       'Label2_CakePerfect', 'Label2_CandyDefected', 'Label2_CandyPerfect',
       'Label2_CansDefected', 'Label2_CansPerfect', 'Label2_CerealDefected',
       'Label2_CerealPerfect', 'Label2_ChipsDefected',
       'Label2_ChipsPackDefected', 'Label2_ChipsPackPerfect',
       'Label2_ChipsPerfect', 'Label2_ChocolateDefected',
       'Label2_ChocolatePerfect', 'Defected', 'Perfect']

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
import torch.nn.functional as F
from PIL import Image
import numpy as np

In [ ]:
class MultiClassifier(nn.Module):
    def __init__(self):
        super(MultiClassifier, self).__init__()
        self.ConvLayer1 = nn.Sequential(
            nn.Conv2d(3, 64, 3), # 3, 256, 256
            nn.MaxPool2d(2), # op: 16, 127, 127
            nn.ReLU(), # op: 64, 127, 127
        )
        self.ConvLayer2 = nn.Sequential(
            nn.Conv2d(64, 128, 3), # 64, 127, 127   
            nn.MaxPool2d(2), #op: 128, 63, 63
            nn.ReLU() # op: 128, 63, 63
        )
        self.ConvLayer3 = nn.Sequential(
            nn.Conv2d(128, 256, 3), # 128, 63, 63
            nn.MaxPool2d(2), #op: 256, 30, 30
            nn.ReLU() #op: 256, 30, 30
        )
        self.ConvLayer4 = nn.Sequential(
            nn.Conv2d(256, 512, 3), # 256, 30, 30
            nn.MaxPool2d(2), #op: 512, 14, 14
            nn.ReLU(), #op: 512, 14, 14
            nn.Dropout(0.2)
        )
        self.Linear1 = nn.Linear(512 * 14 * 14, 1024)
        self.Linear2 = nn.Linear(1024, 256)
        self.Linear3 = nn.Linear(256, 35)
        
        
    def forward(self, x):
        x = self.ConvLayer1(x)
        x = self.ConvLayer2(x)
        x = self.ConvLayer3(x)
        x = self.ConvLayer4(x)
        x = x.view(x.size(0), -1)
        x = self.Linear1(x)
        x = self.Linear2(x)
        x = self.Linear3(x)
        return F.sigmoid(x)

In [ ]:
model_path =  '/content/drive/My Drive/Defect_Detection_Model_50_Epochs'

In [ ]:
model = torch.load(model_path, map_location=torch.device('cpu'))
model = model.eval()

In [ ]:
def get_tensor(img):
    tfms = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor()
    ])
    
    return tfms(Image.open(img)).unsqueeze(0)

In [ ]:
def predict(img, label_lst, model):
    tnsr = get_tensor(img)
    op = model(tnsr)
    op_b = torch.round(op)
    
    op_b_np = torch.Tensor.cpu(op_b).detach().numpy()
    
    preds = np.where(op_b_np == 1)[1]
    
    sigs_op = torch.Tensor.cpu(torch.round((op)*100)).detach().numpy()[0]
    
    o_p = np.argsort(torch.Tensor.cpu(op).detach().numpy())[0][::-1]
    
#     print("Argsort: {}".format(o_p))
#     print("Softmax: {}".format(sigs_op))
    
    
    
#     print(preds)
    
    label = []
    for i in preds:
        label.append(label_lst[i])
        
    arg_s = {}
    for i in o_p:
#         arg_s.append(label_lst[int(i)])
        arg_s[label_lst[int(i)]] = sigs_op[int(i)]
        
    return label, list(arg_s.items())[:10]

In [ ]:
"""the above function to predict the accurately predicted labels and 
also the top 10 labels in a descending order of their probability."""

In [ ]:
predict('Files/4891028707851_camera0-10.jpg', labels, model)    #Beverage > BeveragePerfect

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


(['Perfect'],
 [('Perfect', 67.0),
  ('Label1_Biscuits', 43.0),
  ('Label2_CandyPerfect', 40.0),
  ('Label2_BiscuitPefect', 38.0),
  ('Label1_ChipsPack', 36.0),
  ('Label2_CerealPerfect', 36.0),
  ('Label1_Chips', 35.0),
  ('Defected', 34.0),
  ('Label2_BeverageDefected', 33.0),
  ('Label1_Beverage', 33.0)])

In [ ]:
predict('Files/CHIPS0151.jpg', labels, model)  

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


(['Perfect'],
 [('Perfect', 68.0),
  ('Label1_Biscuits', 43.0),
  ('Label2_CandyPerfect', 39.0),
  ('Label2_BiscuitPefect', 37.0),
  ('Label2_CerealPerfect', 35.0),
  ('Label1_ChipsPack', 35.0),
  ('Label1_Chips', 34.0),
  ('Defected', 32.0),
  ('Label2_BeverageDefected', 32.0),
  ('Label1_CAKE', 32.0)])